# Land relief parameterization in coarser resolution 

This scr{ip}t is used to generate the coarse resolution land relief parameters. The main difference from higher resolution is that the data can fit in RAM of the whole each continent. There the parameters can generate in one shot.

## Part 1: setup and functions

In [2]:
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import numpy as np
from shapely.geometry import Polygon,mapping,box
from shapely import segmentize
import time
import sys
from joblib import Parallel, delayed
from minio import Minio
from eumap.misc import ttprint
import requests
import pandas as pd
import pickle

def find_equi_7_proj(epsg4326_bound,equi_7_proj):
    # Extract coordinates from bounding box
    xmin, ymin, xmax, ymax = epsg4326_bound

    # Define coordinates for the polygon
    polygon_coords = [(xmin, ymin), (xmax, ymin), (xmax, ymax),(xmin, ymax)]

    # Create polygon
    polygon = Polygon(polygon_coords)

    # Create a GeoDataFrame with a single row representing the bounding box
    gdf = gpd.GeoDataFrame(geometry=[Polygon(polygon_coords)])
    # Set CRS for the GeoDataFrame
    gdf.crs = 'EPSG:4326'
    # Check the resulting GeoDataFrame
    gdf.geometry = segmentize(gdf.geometry,max_segment_length=0.000277778)

    return gdf.to_crs(equi_7_proj).geometry.bounds.values[0]

def find_epsg4326_proj(equi7_bound,equi_7_proj):
    # Extract coordinates from bounding box
    xmin, ymin, xmax, ymax = equi7_bound

    # Define coordinates for the polygon
    polygon_coords = [(xmin, ymin), (xmax, ymin), (xmax, ymax),(xmin, ymax)]

    # Create polygon
    polygon = Polygon(polygon_coords)

    # Create a GeoDataFrame with a single row representing the bounding box
    gdf = gpd.GeoDataFrame(geometry=[Polygon(polygon_coords)])
    # Set CRS for the GeoDataFrame
    gdf.crs = equi_7_proj
    gdf.geometry = segmentize(gdf.geometry,max_segment_length=30)
    return gdf.to_crs('EPSG:4326')



equi7_ind={'AF':'+proj=aeqd +lat_0=8.5 +lon_0=21.5 +x_0=5621452.01998 +y_0=5990638.42298 +datum=WGS84 +units=m +no_defs',
'AN':'+proj=aeqd +lat_0=-90 +lon_0=0 +x_0=3714266.97719 +y_0=3402016.50625 +datum=WGS84 +units=m +no_defs',
'AS':'+proj=aeqd +lat_0=47 +lon_0=94 +x_0=4340913.84808 +y_0=4812712.92347 +datum=WGS84 +units=m +no_defs',
'EU':'+proj=aeqd +lat_0=53 +lon_0=24 +x_0=5837287.81977 +y_0=2121415.69617 +datum=WGS84 +units=m +no_defs',
'NA':'+proj=aeqd +lat_0=52 +lon_0=-97.5 +x_0=8264722.17686 +y_0=4867518.35323 +datum=WGS84 +units=m +no_defs',
'OC':'+proj=aeqd +lat_0=-19.5 +lon_0=131.5 +x_0=6988408.5356 +y_0=7654884.53733 +datum=WGS84 +units=m +no_defs',
'SA':'+proj=aeqd +lat_0=-14 +lon_0=-60.5 +x_0=7257179.23559 +y_0=5592024.44605 +datum=WGS84 +units=m +no_defs'}

conts = ['AF',
'AS',
'EU',
'NA',
'OC',
'SA']




In [3]:
# scale factors for each parameters
p_table=pd.read_csv('scaling.csv')
p_table.head()

,parameters,old_data_type,ori_min,ori_max,multiplier,new_data_type,final_min,final_max,no_data
0,geomorphon,Int16,1.000000,10.00000,1,Byte,1.000000,10.000000,255
1,hillshade,Float32,0.000000,28357.00000,1,UInt16,0.000000,28357.000000,65535
2,slope.in.degree,Float32,0.000000,15.29000,100,UInt16,0.000000,1529.000000,65535
3,ls.factor,Float32,0.000000,13.00000,1000,UInt16,0.000000,13000.000000,65535
4,pro.curv,Float32,-8.210329,8.01173,1000,Int16,-8210.329056,8011.730194,32767


## Part 2: parameterization in the for-loop by continents

In [ ]:
#def worker_cont(cont,p_table):
for cont in conts:
    s3_config = {
        'access_key': 's3_key',
        'secret_access_key': 's3_secret_key',
        'host': '{ip}',
        'bucket': 'tmp-global-geomorpho'}
    client = Minio(s3_config['host'], s3_config['access_key'], s3_config['secret_access_key'], secure=False) 

    for resolution in [960,480]:
        url_cont=f'http://{ip}/tmp-global-geomorpho/v4/{cont}/tan.curv_edtm_m_{resolution}m_s_20000101_20221231_{cont.lower().replace("_",".")}_equi7_v20241230.tif'
        r = requests.head(url_cont)
        if r.status_code == 200:
            ttprint(f'{cont} has been process')
            continue
        equi_7_proj=equi7_ind[cont]
        gpd_cont=gpd.read_file(f'Equi7Grid/src/equi7grid/grids/{cont}/PROJ/EQUI7_V14_{cont}_PROJ_ZONE.shp')
        equi7_bound = np.array(gpd_cont.bounds)[0]
        gdalwarp_bbox = ' '.join([str(i) for i in equi7_bound])
        tmp_outdir=f'/tmp/tmp-global-geomorpho/{cont}'
        os.makedirs(tmp_outdir,exist_ok=True)
        gdal_cmd_reproject = f'gdalwarp  -t_srs "{equi_7_proj}" -tr {resolution} {resolution} -r average \
        -co TILED=YES -co BIGTIFF=YES -co COMPRESS=DEFLATE -co ZLEVEL=9 -co BLOCKXSIZE=1024 \
        -co BLOCKYSIZE=1024 -co NUM_THREADS=8 -co SPARSE_OK=TRUE -of GTiff -overwrite'
        url_cont=f'http://{ip}/tmp-global-geomorpho/v4/{cont}/dtm_edtm_m_{resolution}m_s_20000101_20221231_{cont.lower().replace("_",".")}_equi7_v20241230.tif'
        r = requests.head(url_cont)
        if r.status_code == 200:
            ttprint(f'{cont} dtm is on S3')
        else:

            out_ori_file=f'dtm_edtm_m_{resolution}m_s_20000101_20221231_{cont.lower().replace("_",".")}_equi7_v20241230.tif'
            rn_file = f'{tmp_outdir}/{out_ori_file}'
            filepath = '/vsicurl/{ip}/global/edtm/legendtm_rf_30m_m_s_20000101_20231231_go_epsg.4326_v20250130.tif'

            os.system(f'{gdal_cmd_reproject} -te {gdalwarp_bbox} {filepath} {tmp_outdir}/scaled_dtm_tmp.tif')
            os.system(f'gdal_calc.py --overwrite -A {tmp_outdir}/scaled_dtm_tmp.tif \
                    --outfile={rn_file} --calc="A * 0.1" \
                    --type=Float32 --co="COMPRESS=DEFLATE" --co="BLOCKXSIZE=2048" --co="BLOCKYSIZE=2048"')
            s3_path = f"v4/{cont}/{out_ori_file}"
            client.fput_object(s3_config['bucket'], s3_path, rn_file)
            ttprint(f'{ip}/tmp-global-geomorpho/{s3_path} on S3')
        
        gdal_cmd = f'gdalwarp  \
        -co TILED=YES -co BIGTIFF=YES -co COMPRESS=DEFLATE -co ZLEVEL=9 -co BLOCKXSIZE=1024 \
        -co BLOCKYSIZE=1024 -co NUM_THREADS=8 -co SPARSE_OK=TRUE -of GTiff -overwrite'
        input_file = f'{tmp_outdir}/dtm_cont.tif'
        os.system(f'{gdal_cmd} \
                  /vsicurl/http://{ip}/tmp-global-geomorpho/v4/{cont}/dtm_edtm_m_{resolution}m_s_20000101_20221231_{cont.lower().replace("_",".")}_equi7_v20241230.tif \
                  {input_file}')

        import whitebox_workflows
        from whitebox_workflows import download_sample_data, show, WbEnvironment

        # Test your license by setting up the WbW environment
        wbe = whitebox_workflows.WbEnvironment()
        start_time = time.time()
        #tmp_dtm_file=f'vsicrul/http://{ip}/tmp-global-geomorpho/{s3_path}'
        # Reading raster data
        dtm = wbe.read_raster(input_file)
        ttprint("read_raster--- %s seconds ---" % (time.time() - start_time))
        outdir=f'/mnt/{server_name}/tmp-global-geomorpho/{cont}'
        #outdir=f'/mnt/landmark/tmp-global-geomorpho/{cont}'
        os.makedirs(outdir,exist_ok=True)
        file_list=[]


        global_landmask_file='http://{ip}/global/dsm.landmask_ensemble_m_30m_s_20000101_20221231_go_epsg.4326_v4.1.tif'
        #tmp_ori_landmask_file = f'{tmp_outdir}/tmp_landmask.tif'
        tmp_landmask_file = f'{tmp_outdir}/landmask.tif'
        #os.system(f'{gdal_cmd_reproject} -r min {global_landmask_file} {tmp_landmask_file}')
        os.system(f'{gdal_cmd_reproject} -te {gdalwarp_bbox} -r min {global_landmask_file} {tmp_landmask_file}')

        start_time = time.time()
        # Reading raster data
        ttprint(f"{cont} read_raster--- %s seconds ---" % (time.time() - start_time))

        # geomorphon
        tmp_geomorphon_file=input_file.replace('dtm','geomorphon')
        scale=p_table[p_table['parameters']=='geomorphon'].multiplier.iloc[0]

        start_time = time.time()
        geomorphon=wbe.geomorphons(dtm, search_distance=3, 
                                  output_forms=True, analyze_residuals=False)
        wbe.write_raster(geomorphon*scale, tmp_geomorphon_file, compress=True)#, compress=False) # Compression is good, but it is a bit slower so here we won't use it.
        ttprint(f"{cont} calculate geomporphon--- %s seconds ---" % (time.time() - start_time))    
        file_list.append(tmp_geomorphon_file)


        # fill depression for hydrological analysis
        tmp_flled_dtm_file=input_file.replace('dtm','nodepress.dtm')
        scale=p_table[p_table['parameters']=='nodepress.dtm'].multiplier.iloc[0]

        start_time = time.time()
        dtm_no_deps = wbe.breach_depressions_least_cost(dtm, fill_deps=True, flat_increment=0.001)
        wbe.write_raster(dtm_no_deps*scale, tmp_flled_dtm_file, compress=True)#, compress=False) # Compression is good, but it is a bit slower so here we won't use it.
        ttprint(f"{cont} fill depressions--- %s seconds ---" % (time.time() - start_time))    
        file_list.append(tmp_flled_dtm_file)


        # slope for hydrology
        tmp_slope_file=input_file.replace('dtm','slope.in.degree')
        scale=p_table[p_table['parameters']=='slope.in.degree'].multiplier.iloc[0]

        start_time = time.time()
        slope_in_degree = wbe.slope(dtm)
        wbe.write_raster(slope_in_degree*scale, tmp_slope_file, compress=True)#, compress=False) # Compression is good, but it is a bit slower so here we won't use it.
        ttprint(f"{cont} calculate slope--- %s seconds ---" % (time.time() - start_time))
        file_list.append(tmp_slope_file)

        # SCA
        tmp_sca_file=input_file.replace('dtm','spec.catch')
        scale=p_table[p_table['parameters']=='spec.catch'].multiplier.iloc[0]

        start_time = time.time()
        sca = wbe.qin_flow_accumulation(dtm_no_deps, out_type='sca', log_transform=True)
        wbe.write_raster(sca*scale, tmp_sca_file, compress=True)#, compress=False) # Compression is good, but it is a bit slower so here we won't use it.
        ttprint(f"{cont} specific catchment area--- %s seconds ---" % (time.time() - start_time))
        file_list.append(tmp_sca_file)

        # ls factor
        tmp_lsfactor_file=input_file.replace('dtm','ls.factor')
        start_time = time.time()
        scale=p_table[p_table['parameters']=='ls.factor'].multiplier.iloc[0]

        ls_factor=wbe.sediment_transport_index(sca, slope_in_degree, sca_exponent=0.4, slope_exponent=1.3)
        wbe.write_raster(ls_factor*scale, tmp_lsfactor_file, compress=True)#, compress=False) # Compression is good, but it is a bit slower so here we won't use it.
        ttprint(f"{cont} calculate ls factor--- %s seconds ---" % (time.time() - start_time))    
        file_list.append(tmp_lsfactor_file)

        #twi
        start_time = time.time()
        tmp_twi_file=input_file.replace('dtm','twi')
        scale=p_table[p_table['parameters']=='twi'].multiplier.iloc[0]

        twi = wbe.wetness_index(specific_catchment_area=sca, slope=slope_in_degree)
        #twi_filled = wbe.fill_missing_data(twi, exclude_edge_nodata=True)
        ttprint(f"{cont} topographic wetness index--- %s seconds ---" % (time.time() - start_time))
        wbe.write_raster(twi*scale, tmp_twi_file, compress=True) # Compression is good, but it is a bit slower so here we won't use it.
        file_list.append(tmp_twi_file)

        # Reading raster data
        start_time = time.time()
        # Reading raster data
        ttprint(f"{cont} read local surface raster--- %s seconds ---" % (time.time() - start_time))

        # diff from mean elev
        start_time = time.time()
        tmp_dfme_file=input_file.replace('dtm','dfme')
        scale=p_table[p_table['parameters']=='dfme'].multiplier.iloc[0]

        dfme=wbe.difference_from_mean_elevation(
            dtm, 
            filter_size_x=3, 
            filter_size_y=3)

        wbe.write_raster(dfme*scale, tmp_dfme_file, compress=True) # Compression is good, but it is a bit slower so here we won't use it.
        ttprint(f"{cont} diff from mean elev--- %s seconds ---" % (time.time() - start_time))
        file_list.append(tmp_dfme_file)

        #Spherical Std Dev Of Normals
        start_time = time.time()
        tmp_ssdon_file=input_file.replace('dtm','ssdon')
        scale=p_table[p_table['parameters']=='ssdon'].multiplier.iloc[0]

        start_time = time.time()
        ssdon=wbe.spherical_std_dev_of_normals(
            dtm, 
            filter_size=3 
        )

        wbe.write_raster(ssdon*scale, tmp_ssdon_file, compress=True) # Compression is good, but it is a bit slower so here we won't use it.
        ttprint(f"{cont} spherical std dev of normals--- %s seconds ---" % (time.time() - start_time))
        file_list.append(tmp_ssdon_file)


        # Hillshade
        tmp_hillshade_file=input_file.replace('dtm','hillshade')
        scale=p_table[p_table['parameters']=='hillshade'].multiplier.iloc[0]

        start_time = time.time()
        hs = wbe.multidirectional_hillshade(dtm)
        wbe.write_raster(hs*scale, tmp_hillshade_file, compress=True)#, compress=False) # Compression is good, but it is a bit slower so here we won't use it.
        ttprint(f"{cont} calculate hillshade--- %s seconds ---" % (time.time() - start_time))    
        file_list.append(tmp_hillshade_file)

        # Minic
        tmp_minic_file=input_file.replace('dtm','minic')
        scale=p_table[p_table['parameters']=='minic'].multiplier.iloc[0]

        start_time = time.time()
        minic = wbe.minimal_curvature(dtm, log_transform=True)
        wbe.write_raster(minic*scale, tmp_minic_file, compress=True)#, compress=False) # Compression is good, but it is a bit slower so here we won't use it.
        ttprint(f"{cont} calculate minic--- %s seconds ---" % (time.time() - start_time))
        file_list.append(tmp_minic_file)


        # Maxic
        tmp_maxic_file=input_file.replace('dtm','maxic')
        scale=p_table[p_table['parameters']=='maxic'].multiplier.iloc[0]

        start_time = time.time()
        maxic = wbe.maximal_curvature(dtm, log_transform=True)
        wbe.write_raster(maxic*scale, tmp_maxic_file, compress=True)#, compress=False) # Compression is good, but it is a bit slower so here we won't use it.
        ttprint(f"{cont} calculate maxic--- %s seconds ---" % (time.time() - start_time))    
        file_list.append(tmp_maxic_file)

        # Openness
        tmp_pos_file=input_file.replace('dtm','pos.openness')
        tmp_neg_file=input_file.replace('dtm','neg.openness')
        start_time = time.time()
        pos,neg = wbe.openness(dtm,dist=3)
        scale=p_table[p_table['parameters']=='pos.openness'].multiplier.iloc[0]
        wbe.write_raster(pos*scale, tmp_pos_file, compress=True)#, compress=False) # Compression is good, but it is a bit slower so here we won't use it.
        scale=p_table[p_table['parameters']=='neg.openness'].multiplier.iloc[0]

        wbe.write_raster(neg*scale, tmp_neg_file, compress=True)#, compress=False) # Compression is good, but it is a bit slower so here we won't use it.

        ttprint(f"{cont} calculate openness--- %s seconds ---" % (time.time() - start_time))    
        file_list.append(tmp_pos_file)
        file_list.append(tmp_neg_file)


        # profile curve
        start_time = time.time()
        tmp_procurv_file=input_file.replace('dtm','pro.curv')
        scale=p_table[p_table['parameters']=='pro.curv'].multiplier.iloc[0]
        procurv = wbe.profile_curvature(dtm, log_transform=True)

        wbe.write_raster(procurv*scale, tmp_procurv_file, compress=True) # Compression is good, but it is a bit slower so here we won't use it.

        ttprint(f"{cont} profile curve --- %s seconds ---" % (time.time() - start_time))
        file_list.append(tmp_procurv_file)


        # shape index
        start_time = time.time()
        tmp_shpindx_file=input_file.replace('dtm','shpindx')
        scale=p_table[p_table['parameters']=='shpindx'].multiplier.iloc[0]


        shpindx=wbe.shape_index(dtm)
        wbe.write_raster(shpindx*scale, tmp_shpindx_file, compress=True)
        ttprint(f"{cont} shape index --- %s seconds ---" % (time.time() - start_time))
        file_list.append(tmp_shpindx_file)

        # ring curvature
        start_time = time.time()
        tmp_ring_curv_file=input_file.replace('dtm','ring.curv')
        scale=p_table[p_table['parameters']=='ring.curv'].multiplier.iloc[0]


        ring_curv=wbe.ring_curvature(dtm, log_transform=True)
        wbe.write_raster(ring_curv*scale, tmp_ring_curv_file, compress=True) # Compression is good, but it is a bit slower so here we won't use it.

        ttprint(f"{cont} ring curvature --- %s seconds ---" % (time.time() - start_time))
        file_list.append(tmp_ring_curv_file)

        # tangential curvatures
        start_time = time.time()
        tmp_tan_curv_file=input_file.replace('dtm','tan.curv')
        scale=p_table[p_table['parameters']=='tan.curv'].multiplier.iloc[0]

        tan_curv=wbe.tangential_curvature(dtm, log_transform=True)
        wbe.write_raster(tan_curv*scale, tmp_tan_curv_file, compress=True) # Compression is good, but it is a bit slower so here we won't use it.

        ttprint(f"{cont} tangential curvature --- %s seconds ---" % (time.time() - start_time))
        file_list.append(tmp_tan_curv_file)


        start_time = time.time()
        def para_gdal_warp(file_path,cont,bbox,p_table,tmp_landmask_file):
            file_name = file_path.split('/')[-1]
            parameter = file_name.split('_')[0]
            dtype=p_table[p_table['parameters']==parameter].new_data_type.iloc[0]
            no_data=p_table[p_table['parameters']==parameter].no_data.iloc[0]

            gdalcmd = f'gdalwarp -overwrite -ot {dtype} -tr {resolution} {resolution} -te {bbox} -co TILED=YES -co BIGTIFF=YES -co COMPRESS=DEFLATE -co ZLEVEL=9 -co BLOCKXSIZE=2048 -co BLOCKYSIZE=2048 -co NUM_THREADS=8 -co SPARSE_OK=TRUE'

            file_name = parameter + '_edtm' + '_m' + f'_{resolution}m' + '_s' + '_20000101_20221231' + f'_{cont.lower()}'  + '_equi7' + '_v20241230' + '.tif'
            out_path = f'{outdir}/{file_name}'  
            tmp_out_path = f'{outdir}/tmp_{file_name}'
            os.system(f'{gdalcmd} {file_path} {tmp_out_path}')
            # landmasking
            os.system(f'gdal_calc.py -A {tmp_out_path} -B {tmp_landmask_file} --overwrite --outfile={out_path} \
                        --calc="(B==100)*A + (B!=100)*{no_data}" --type={dtype} --co="ZLEVEL=9" --co="COMPRESS=DEFLATE" \
                        --co="BLOCKXSIZE=2048" --NoDataValue={no_data} --co="BLOCKYSIZE=2048" \
                        --co="NUM_THREADS=8" --co="SPARSE_OK=TRUE"')
            os.remove(file_path)
            return out_path,file_name

        args = [(i,cont,gdalwarp_bbox,p_table,tmp_landmask_file) for i in file_list]
        for arg in args:
            out_file,rn_file=para_gdal_warp(arg[0],arg[1],arg[2],arg[3],arg[4])
            s3_path = f"v4/{cont}/{rn_file}"
            client.fput_object(s3_config['bucket'], s3_path, out_file)
            os.remove(out_file)
            ttprint(f'http://{ip}/tmp-global-geomorpho/{s3_path} on S3')
        #os.remove(rn_file)
        os.system(f'rm -r {tmp_outdir}/*')
        ttprint(f"{cont} crop and save to local--- %s seconds ---" % (time.time() - start_time))


Parallel(n_jobs=6)(delayed(worker_cont)(i,p_table) for i in conts)